In [177]:
import pandas as pd
import numpy as np # type: ignore
from pybaseball import *
import statsapi
import os

from pull_data import *
%load_ext autoreload
%autoreload 2
import requests

from pybaseball import cache
cache.enable()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [178]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [179]:
from datetime import date

# Get today's date in YYYY-MM-DD format
days_offset = 0
today = (date.today() + pd.DateOffset(days=days_offset)).strftime('%Y-%m-%d')

# Get today's schedule
games = statsapi.schedule(start_date=today, end_date=today)

# Loop through games and extract probable pitchers
for game in games:
    home_team = game['home_name']
    away_team = game['away_name']
    home_pitcher = game.get('home_probable_pitcher', 'TBD')
    away_pitcher = game.get('away_probable_pitcher', 'TBD')
    
    print(f"{away_team} ({away_pitcher}) @ {home_team} ({home_pitcher})")

Cincinnati Reds (Nick Lodolo) @ Chicago Cubs (Drew Pomeranz)
Athletics (Gunnar Hoglund) @ Toronto Blue Jays (Braydon Fisher)
St. Louis Cardinals (Sonny Gray) @ Texas Rangers (Patrick Corbin)
Chicago White Sox (Davis Martin) @ Baltimore Orioles (Dean Kremer)
Milwaukee Brewers (Rob Zastryzny) @ Philadelphia Phillies (Jesús Luzardo)
Los Angeles Angels (Kyle Hendricks) @ Cleveland Guardians (Slade Cecconi)
Detroit Tigers (Tarik Skubal) @ Kansas City Royals (Michael Wacha)
Colorado Rockies (Antonio Senzatela) @ New York Mets (Kodai Senga)
Tampa Bay Rays (Zack Littell) @ Houston Astros (Colton Gordon)
Boston Red Sox (Walker Buehler) @ Atlanta Braves (Spencer Schwellenbach)
San Francisco Giants (Robbie Ray) @ Miami Marlins (Edward Cabrera)
Minnesota Twins (Bailey Ober) @ Seattle Mariners (Bryce Miller)
New York Yankees (Will Warren) @ Los Angeles Dodgers (Landon Knack)
Pittsburgh Pirates (Bailey Falter) @ San Diego Padres (Dylan Cease)
Washington Nationals (Michael Soroka) @ Arizona Diamondba

In [180]:
def spray_angle(hc_x, hc_y, stand):
    mult = 1
    if stand == 'L':
        mult = -1
    return mult * np.degrees(np.arctan2((hc_x - 125.42), (198.27 - hc_y)))

In [181]:
import os
api_key = os.getenv("api_key")
print(api_key)

fa1d935c1d67073458a1f970bc434075


In [182]:
# Note that BBE = attempts = number of events
pitcher_barrels = statcast_pitcher_exitvelo_barrels(2025)

In [183]:
start_date = (date.today() + pd.DateOffset(days=-30)).strftime('%Y-%m-%d')
end_date = today
pitcher_ids = get_starting_ids(days_offset=days_offset)
pitcher_id_nos = [p[0] for p in pitcher_ids]
starters_data = {}
pitchers_arsenals = {}
pitcher_opponents = {}
pitcher_games = {}
for id, opposing_team, game_id in pitcher_ids:
    pitcher_data = statcast_pitcher(start_dt=start_date, end_dt=end_date,player_id=id)
    pitcher_opponents[id] = opposing_team
    starters_data[id] = pitcher_data.copy()
    pitcher_games[id] = game_id
    pitchers_arsenals[id] = {'R': get_pitch_arsenal(pitcher_data, 'R'), 'L': get_pitch_arsenal(pitcher_data, 'L')}

['Drew Pomeranz', 'Nick Lodolo', 'Braydon Fisher', 'Gunnar Hoglund', 'Patrick Corbin', 'Sonny Gray', 'Dean Kremer', 'Davis Martin', 'Jesús Luzardo', 'Rob Zastryzny', 'Slade Cecconi', 'Kyle Hendricks', 'Michael Wacha', 'Tarik Skubal', 'Kodai Senga', 'Antonio Senzatela', 'Colton Gordon', 'Zack Littell', 'Spencer Schwellenbach', 'Walker Buehler', 'Edward Cabrera', 'Robbie Ray', 'Bryce Miller', 'Bailey Ober', 'Landon Knack', 'Will Warren', 'Dylan Cease', 'Bailey Falter', 'Brandon Pfaadt', 'Michael Soroka']
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gatherin

In [184]:
# pitcher_barrels_copy = pitcher_barrels[pitcher_barrels['attempts'] > 50]
pitcher_barrels_copy = pitcher_barrels[pitcher_barrels['player_id'].isin(pitcher_id_nos)].copy()
pitcher_barrels_copy['barrel_percentage'] = pitcher_barrels_copy['player_id'].apply(get_barrel_percentage, start_date=start_date, end_date=end_date)
pitcher_barrels_copy['left_barrel_pct'] = pitcher_barrels_copy['barrel_percentage'].apply(lambda x: x['left_pct'])
pitcher_barrels_copy['right_barrel_pct'] = pitcher_barrels_copy['barrel_percentage'].apply(lambda x: x['right_pct'])
pitcher_barrels_copy['opponent'] = pitcher_barrels_copy['player_id'].apply(lambda x: pitcher_opponents[x])
pitcher_barrels_copy['game_id'] = pitcher_barrels_copy['player_id'].apply(lambda x: pitcher_games.get(x))
pitcher_barrels_copy[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'opponent']]

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,opponent
0,"Senzatela, Antonio",622608,777700,214,22,10.30,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.07,0.10,home
3,"Martin, Davis",663436,777699,204,19,9.30,"{'left_barrels': 2, 'right_barrels': 8, 'total...",0.06,0.13,home
4,"Kremer, Dean",665152,777699,204,19,9.30,"{'left_barrels': 3, 'right_barrels': 2, 'total...",0.07,0.04,away
5,"Schwellenbach, Spencer",680885,777706,202,14,6.90,"{'left_barrels': 5, 'right_barrels': 2, 'total...",0.10,0.05,away
8,"Wacha, Michael",608379,777694,196,13,6.60,"{'left_barrels': 3, 'right_barrels': 3, 'total...",0.07,0.05,away
9,"Lodolo, Nick",666157,777704,194,15,7.70,"{'left_barrels': 1, 'right_barrels': 7, 'total...",0.04,0.11,home
10,"Pfaadt, Brandon",694297,777696,193,26,13.50,"{'left_barrels': 10, 'right_barrels': 7, 'tota...",0.21,0.17,away
14,"Littell, Zack",641793,777693,189,27,14.30,"{'left_barrels': 3, 'right_barrels': 10, 'tota...",0.08,0.17,home
15,"Ober, Bailey",641927,777695,189,15,7.90,"{'left_barrels': 3, 'right_barrels': 4, 'total...",0.07,0.11,home
29,"Falter, Bailey",663559,777691,176,14,8.00,"{'left_barrels': 0, 'right_barrels': 4, 'total...",0.00,0.07,home


In [185]:
filtered_pitcher_barrels_copy = pitcher_barrels_copy[(pitcher_barrels_copy['left_barrel_pct'] > 0.07) | (pitcher_barrels_copy['right_barrel_pct'] > 0.07)].copy()
filtered_pitcher_barrels_copy['sides'] = filtered_pitcher_barrels_copy.apply(lambda row: get_sides(row['left_barrel_pct'], row['right_barrel_pct']),axis=1)
filtered_pitcher_barrels_copy['arsenal'] = filtered_pitcher_barrels_copy.apply(lambda row: pitchers_arsenals[row['player_id']], axis=1)
filtered_pitcher_barrels_copy[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent']]

,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent
0,"Senzatela, Antonio",622608,777700,214,22,10.30,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.07,0.10,[R],"{'R': {'FF': 48.4, 'SL': 32.8}, 'L': {'FF': 52...",home
3,"Martin, Davis",663436,777699,204,19,9.30,"{'left_barrels': 2, 'right_barrels': 8, 'total...",0.06,0.13,[R],"{'R': {'FC': 44.516129032258064, 'FF': 31.6129...",home
4,"Kremer, Dean",665152,777699,204,19,9.30,"{'left_barrels': 3, 'right_barrels': 2, 'total...",0.07,0.04,[L],"{'R': {'FC': 43.58974358974359, 'SI': 31.62393...",away
5,"Schwellenbach, Spencer",680885,777706,202,14,6.90,"{'left_barrels': 5, 'right_barrels': 2, 'total...",0.10,0.05,[L],"{'R': {'FF': 32.743362831858406, 'SL': 28.3185...",away
9,"Lodolo, Nick",666157,777704,194,15,7.70,"{'left_barrels': 1, 'right_barrels': 7, 'total...",0.04,0.11,[R],"{'R': {'FF': 31.52173913043478, 'CH': 27.17391...",home
10,"Pfaadt, Brandon",694297,777696,193,26,13.50,"{'left_barrels': 10, 'right_barrels': 7, 'tota...",0.21,0.17,"[L, R]","{'R': {'ST': 40.27777777777778, 'FF': 30.09259...",away
14,"Littell, Zack",641793,777693,189,27,14.30,"{'left_barrels': 3, 'right_barrels': 10, 'tota...",0.08,0.17,"[L, R]","{'R': {'SL': 31.34328358208955, 'FF': 26.86567...",home
15,"Ober, Bailey",641927,777695,189,15,7.90,"{'left_barrels': 3, 'right_barrels': 4, 'total...",0.07,0.11,[R],"{'R': {'FF': 32.33830845771144, 'CH': 25.87064...",home
42,"Gray, Sonny",543243,777697,171,17,9.90,"{'left_barrels': 4, 'right_barrels': 5, 'total...",0.09,0.16,"[L, R]","{'R': {'ST': 27.84090909090909, 'SI': 23.86363...",home
45,"Hendricks, Kyle",543294,777703,171,17,9.90,"{'left_barrels': 6, 'right_barrels': 6, 'total...",0.12,0.12,"[L, R]","{'R': {'SI': 39.83050847457627, 'CH': 30.93220...",home


Next step is
- For each batter in a lineup that a pitcher is facing, we want to get their barrel % against the pitcher's arsenal (the pitchers > 18%)
- Calculate the weighted pull air percentage (pull % * air %)
- Also eventually can add blast %, iso

In [186]:
pitcher_df = filtered_pitcher_barrels_copy.copy()
pitcher_df['opposing_lineup'] = pitcher_df.apply(lambda row: get_opposing_lineup(row['game_id'], row['opponent']), axis=1)
pitcher_df[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent', 'opposing_lineup']]

Error fetching lineup for game 777700: list index out of range
Error fetching lineup for game 777699: list index out of range
Error fetching lineup for game 777699: list index out of range
Error fetching lineup for game 777706: list index out of range
Error fetching lineup for game 777704: list index out of range
Error fetching lineup for game 777696: list index out of range
Error fetching lineup for game 777693: list index out of range
Error fetching lineup for game 777695: list index out of range
Error fetching lineup for game 777697: list index out of range
Error fetching lineup for game 777703: list index out of range
Error fetching lineup for game 777691: list index out of range
Error fetching lineup for game 777700: list index out of range
Error fetching lineup for game 777697: list index out of range
Error fetching lineup for game 777698: list index out of range
Error fetching lineup for game 777706: list index out of range
Error fetching lineup for game 777695: list index out o

,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent,opposing_lineup
0,"Senzatela, Antonio",622608,777700,214,22,10.30,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.07,0.10,[R],"{'R': {'FF': 48.4, 'SL': 32.8}, 'L': {'FF': 52...",home,None
3,"Martin, Davis",663436,777699,204,19,9.30,"{'left_barrels': 2, 'right_barrels': 8, 'total...",0.06,0.13,[R],"{'R': {'FC': 44.516129032258064, 'FF': 31.6129...",home,None
4,"Kremer, Dean",665152,777699,204,19,9.30,"{'left_barrels': 3, 'right_barrels': 2, 'total...",0.07,0.04,[L],"{'R': {'FC': 43.58974358974359, 'SI': 31.62393...",away,None
5,"Schwellenbach, Spencer",680885,777706,202,14,6.90,"{'left_barrels': 5, 'right_barrels': 2, 'total...",0.10,0.05,[L],"{'R': {'FF': 32.743362831858406, 'SL': 28.3185...",away,None
9,"Lodolo, Nick",666157,777704,194,15,7.70,"{'left_barrels': 1, 'right_barrels': 7, 'total...",0.04,0.11,[R],"{'R': {'FF': 31.52173913043478, 'CH': 27.17391...",home,None
10,"Pfaadt, Brandon",694297,777696,193,26,13.50,"{'left_barrels': 10, 'right_barrels': 7, 'tota...",0.21,0.17,"[L, R]","{'R': {'ST': 40.27777777777778, 'FF': 30.09259...",away,None
14,"Littell, Zack",641793,777693,189,27,14.30,"{'left_barrels': 3, 'right_barrels': 10, 'tota...",0.08,0.17,"[L, R]","{'R': {'SL': 31.34328358208955, 'FF': 26.86567...",home,None
15,"Ober, Bailey",641927,777695,189,15,7.90,"{'left_barrels': 3, 'right_barrels': 4, 'total...",0.07,0.11,[R],"{'R': {'FF': 32.33830845771144, 'CH': 25.87064...",home,None
42,"Gray, Sonny",543243,777697,171,17,9.90,"{'left_barrels': 4, 'right_barrels': 5, 'total...",0.09,0.16,"[L, R]","{'R': {'ST': 27.84090909090909, 'SI': 23.86363...",home,None
45,"Hendricks, Kyle",543294,777703,171,17,9.90,"{'left_barrels': 6, 'right_barrels': 6, 'total...",0.12,0.12,"[L, R]","{'R': {'SI': 39.83050847457627, 'CH': 30.93220...",home,None


In [187]:
hitter_barrels = statcast_batter_exitvelo_barrels(2025)

In [188]:
def get_hitter_barrels(hitter_barrels, hitter_ids, start_date, end_date, pitch_mix, sides):
    if hitter_ids is None or len(hitter_ids) == 0:
        return {}
    hitter_barrels_copy = hitter_barrels[hitter_barrels['player_id'].isin(hitter_ids)].copy()
    hitter_barrels_copy['barrel_data'] = hitter_barrels_copy['player_id'].apply(hitter_barrel_percentage, start_date=start_date, end_date=end_date, pitch_mix=pitch_mix, sides=sides)
    hitter_barrels_copy['barrel_percentage'] = hitter_barrels_copy.apply(lambda row: get_barrel_pcts(row['barrel_data']), axis=1)
    hitter_barrels_copy['barrel_qual'] = hitter_barrels_copy['barrel_percentage'].apply(lambda x: check_barrel_pcts(x))
    # print(hitter_barrels_copy['barrel_qual'])
    filtered = hitter_barrels_copy[hitter_barrels_copy['barrel_qual'].apply(lambda x: x != [0,0])].copy()
    # print(filtered)
    if len(filtered) > 3:
        # Create a new column with just the first element
        filtered['barrel_qual_first'] = filtered['barrel_qual'].apply(lambda x: x[0])
        # Sort by that column and get top 3
        top_3 = filtered.nlargest(3, 'barrel_qual_first')
        # Drop the temporary column if you don't need it
        top_3 = top_3.drop('barrel_qual_first', axis=1)
        return dict(zip(top_3['last_name, first_name'], top_3['barrel_qual']))
    else:
        return dict(zip(filtered['last_name, first_name'], filtered['barrel_qual']))

In [189]:
hitter_pitcher_df = pitcher_df.copy()
hitter_pitcher_df['hitter_barrels'] = hitter_pitcher_df.apply(lambda row: get_hitter_barrels(hitter_barrels, row['opposing_lineup'], start_date, end_date, row['arsenal'], row['sides']), axis=1)
hitter_pitcher_df['num_hitters'] = hitter_pitcher_df['hitter_barrels'].apply(lambda x: len(x))
hitter_pitcher_df[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent', 'opposing_lineup', 'hitter_barrels', 'num_hitters']]

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent,opposing_lineup,hitter_barrels,num_hitters
0,"Senzatela, Antonio",622608,777700,214,22,10.30,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.07,0.10,[R],"{'R': {'FF': 48.4, 'SL': 32.8}, 'L': {'FF': 52...",home,None,{},0
3,"Martin, Davis",663436,777699,204,19,9.30,"{'left_barrels': 2, 'right_barrels': 8, 'total...",0.06,0.13,[R],"{'R': {'FC': 44.516129032258064, 'FF': 31.6129...",home,None,{},0
4,"Kremer, Dean",665152,777699,204,19,9.30,"{'left_barrels': 3, 'right_barrels': 2, 'total...",0.07,0.04,[L],"{'R': {'FC': 43.58974358974359, 'SI': 31.62393...",away,None,{},0
5,"Schwellenbach, Spencer",680885,777706,202,14,6.90,"{'left_barrels': 5, 'right_barrels': 2, 'total...",0.10,0.05,[L],"{'R': {'FF': 32.743362831858406, 'SL': 28.3185...",away,None,{},0
9,"Lodolo, Nick",666157,777704,194,15,7.70,"{'left_barrels': 1, 'right_barrels': 7, 'total...",0.04,0.11,[R],"{'R': {'FF': 31.52173913043478, 'CH': 27.17391...",home,None,{},0
10,"Pfaadt, Brandon",694297,777696,193,26,13.50,"{'left_barrels': 10, 'right_barrels': 7, 'tota...",0.21,0.17,"[L, R]","{'R': {'ST': 40.27777777777778, 'FF': 30.09259...",away,None,{},0
14,"Littell, Zack",641793,777693,189,27,14.30,"{'left_barrels': 3, 'right_barrels': 10, 'tota...",0.08,0.17,"[L, R]","{'R': {'SL': 31.34328358208955, 'FF': 26.86567...",home,None,{},0
15,"Ober, Bailey",641927,777695,189,15,7.90,"{'left_barrels': 3, 'right_barrels': 4, 'total...",0.07,0.11,[R],"{'R': {'FF': 32.33830845771144, 'CH': 25.87064...",home,None,{},0
42,"Gray, Sonny",543243,777697,171,17,9.90,"{'left_barrels': 4, 'right_barrels': 5, 'total...",0.09,0.16,"[L, R]","{'R': {'ST': 27.84090909090909, 'SI': 23.86363...",home,None,{},0
45,"Hendricks, Kyle",543294,777703,171,17,9.90,"{'left_barrels': 6, 'right_barrels': 6, 'total...",0.12,0.12,"[L, R]","{'R': {'SI': 39.83050847457627, 'CH': 30.93220...",home,None,{},0


In [193]:
# Barrel percent, blast percent
all_keys = {}
for row in hitter_pitcher_df['hitter_barrels']:
    for key in row:
        all_keys[key] = row[key]
sorted_batters = all_keys.items()
sorted_batters = sorted(sorted_batters, key=lambda x: x[1], reverse=True)
for batter in sorted_batters:
    print(batter)

('Springer, George', [np.float64(0.23333333333333334), np.float64(0.3333333333333333)])
('Witt Jr., Bobby', [np.float64(0.1744186046511628), np.float64(0.38372093023255816)])
('Bichette, Bo', [np.float64(0.16853932584269662), np.float64(0.25842696629213485)])
('Guerrero Jr., Vladimir', [np.float64(0.15476190476190477), np.float64(0.38095238095238093)])
('Perez, Salvador', [np.float64(0.125), np.float64(0.3472222222222222)])
